<a href="https://colab.research.google.com/github/harshavardhinisri/CS-6320-NLP/blob/main/nlp_a1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip A1_DATASET.zip

Archive:  A1_DATASET.zip
   creating: A1_DATASET/
  inflating: A1_DATASET/.DS_Store    
  inflating: __MACOSX/A1_DATASET/._.DS_Store  
  inflating: A1_DATASET/train.txt    
  inflating: __MACOSX/A1_DATASET/._train.txt  
  inflating: A1_DATASET/val.txt      
  inflating: __MACOSX/A1_DATASET/._val.txt  


In [ ]:
!head A1_DATASET/train.txt

I booked two rooms four months in advance at the Talbott . We were placed on the top floor next to the elevators , which are used all night long . When speaking to the front desk , I was told that they were simply honoring my request for an upper floor , which I had requested for a better view . I am looking at a brick wall , and getting no sleep . He also told me that they had received complaints before from guests on the 16th floor , and were aware of the noise problem . Why then did they place us on this floor when the hotel is not totally booked ? A request for an upper floor does not constitute placing someone on the TOP floor and using that request to justify this . If you decide to stay here , request a room on a lower floor and away from the elevator ! I spoke at length when booking my two rooms about my preferences . This is simply poor treatment of a guest whom they believed would not complain .
I LOVED this hotel . The room was so chic and trendy , the bed was comfortable , 

In [ ]:
!head A1_DATASET/val.txt

I stayed for four nights while attending a conference . The hotel is in a great spot - easy walk to Michigan Ave shopping or Rush St. , but just off the busy streets . The room I had was spacious , and very well-appointed . The staff was friendly , and the fitness center , while not huge , was well-equipped and clean . I 've stayed at a number of hotels in Chicago , and this one is my favorite . Internet was n't free , but at $ 10 for 24 hours is cheaper than most business hotels , and it worked very well .
we love the location and proximity to everything . The staff was very friendly and courteous . They were so nice to our 2.5 year old boy . got his backpack full of goodies the moment we arrived . We got free wifi and morning drinks by signing up for select guest program . Ca n't beat that ! the only minor issue is the elevator . we have to take 2 separate elevator trips to get to our room . It got a little annoying when we were going in and out often . Otherwise , it was a great sta

**Unigram and Bigram without smoothening**

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
from collections import Counter
nltk.download('stopwords')

def preprocess(text):
  text = text.lower()
  translator = str.maketrans('', '', string.punctuation)
  text = text.translate(translator)
  tokens = text.split()
  tokens = [token for token in tokens if not token.isdigit()]
  stemmer = PorterStemmer()
  tokens = [stemmer.stem(word) for word in tokens]
  return tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def calculate_unigram_counts(sentences):
    unigram_counts = Counter()

    for sentence in sentences:
        for token in sentence:
            unigram_counts[token] += 1

    return unigram_counts

def calculate_bigram_counts(sentences):
    bigram_counts = Counter()

    for sentence in sentences:
        for i in range(len(sentence) - 1):
            bigram = (sentence[i], sentence[i + 1])
            bigram_counts[bigram] += 1

    return bigram_counts

In [ ]:
def train_ngram_model(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    sentences = [preprocess(line) for line in lines]

    unigram_counts = calculate_unigram_counts(sentences)
    bigram_counts = calculate_bigram_counts(sentences)

    total_unigrams = sum(unigram_counts.values())

    unigram_probs = {token: count / total_unigrams for token, count in unigram_counts.items()}
    bigram_probs = {(w1, w2): count / unigram_counts[w1] for (w1, w2), count in bigram_counts.items()}

    return unigram_probs, bigram_probs

filename = "A1_DATASET/train.txt"
unigram_probs, bigram_probs = train_ngram_model(filename)

In [ ]:
import math

def compute_perplexity_unigram(sentences, unigram_probs):
    N = sum([len(sentence) for sentence in sentences])
    log_prob = sum([math.log(unigram_probs.get(word, 1e-10)) for sentence in sentences for word in sentence])
    return math.exp(-log_prob/N)

def compute_perplexity_bigram(sentences, unigram_counts, bigram_probs):
    N = sum([len(sentence) for sentence in sentences]) - len(sentences)
    total_log_prob = 0
    for sentence in sentences:
        for i in range(1, len(sentence)):
            bigram = (sentence[i-1], sentence[i])
            bigram_prob = bigram_probs.get(bigram, (1 / len(unigram_counts)))
            total_log_prob += math.log(bigram_prob)
    return math.exp(-total_log_prob/N)


In [ ]:
# Preprocess the data
with open("A1_DATASET/train.txt", 'r') as file:
    dev_lines = file.readlines()
sentences_dev = [preprocess(line) for line in dev_lines]

# Compute perplexities
unigram_counts = calculate_unigram_counts(sentences_dev)
perplexity_unigram = compute_perplexity_unigram(sentences_dev, unigram_probs)
perplexity_bigram = compute_perplexity_bigram(sentences_dev, unigram_counts, bigram_probs)

print(f"Unigram Perplexity without smoothening for train.txt: {perplexity_unigram}")
print(f"Bigram Perplexity without smoothening for train.txt: {perplexity_bigram}")

Unigram Perplexity without smoothening for train.txt: 420.36173656451945
Bigram Perplexity without smoothening for train.txt: 33.692191237035296


In [ ]:
# Preprocess the data
with open("A1_DATASET/val.txt", 'r') as file:
    dev_lines = file.readlines()
sentences_dev = [preprocess(line) for line in dev_lines]

# Compute perplexities
unigram_counts = calculate_unigram_counts(sentences_dev)
perplexity_unigram = compute_perplexity_unigram(sentences_dev, unigram_probs)
perplexity_bigram = compute_perplexity_bigram(sentences_dev, unigram_counts, bigram_probs)

print(f"Unigram Perplexity without smoothening for val.txt: {perplexity_unigram}")
print(f"Bigram Perplexity without smoothening for val.txt: {perplexity_bigram}")

Unigram Perplexity without smoothening for val.txt: 560.434232044935
Bigram Perplexity without smoothening for val.txt: 110.84972416063393


**Unigram and Bigram with K-smoothening and Laplace smootheing**

In [ ]:
def handle_oov(sentences):
    all_words = [word for sentence in sentences for word in sentence]
    word_freq = Counter(all_words)

    sentences_with_oov = []
    for sentence in sentences:
        new_sentence = [word if word_freq[word] > 1 else '<OOV>' for word in sentence]
        sentences_with_oov.append(new_sentence)

    return sentences_with_oov

In [ ]:
def train_ngram_model(filename,k):
    with open(filename, 'r') as file:
        lines = file.readlines()

    sentences = [preprocess(line) for line in lines]

    sentences = handle_oov(sentences)

    unigram_counts = calculate_unigram_counts(sentences)
    bigram_counts = calculate_bigram_counts(sentences)

    total_unigrams = sum(unigram_counts.values())
    V = len(unigram_counts)


    unigram_probs = {token: (count + k) / (total_unigrams + k*V) for token, count in unigram_counts.items()}
    bigram_probs = {(w1, w2): (count + k) / (unigram_counts[w1] + k*V) for (w1, w2), count in bigram_counts.items()}

    return unigram_probs, bigram_probs, V

In [ ]:
filename = "A1_DATASET/train.txt"
unigram_probs_laplace, bigram_probs_laplace, V = train_ngram_model(filename,1)
unigram_probs_kSmooth, bigram_probs_kSmooth, V = train_ngram_model(filename,.01)
unigram_probs_kSmooth_0_5, bigram_probs_kSmooth_0_5, V = train_ngram_model(filename,.5)

In [ ]:
def compute_perplexity_unigram(sentences, unigram_probs):
  N = sum([len(sentence) for sentence in sentences])
  log_prob = sum([math.log(unigram_probs.get(word, 1e-10)) for sentence in sentences for word in sentence])
  return math.exp(-log_prob/N)


def compute_perplexity_bigram(sentences, unigram_probs, bigram_probs, V):
    N = sum([len(sentence) for sentence in sentences]) - len(sentences)
    total_log_prob = 0
    for sentence in sentences:
        for i in range(1, len(sentence)):
            bigram = (sentence[i-1], sentence[i])
            if bigram in bigram_probs:
                bigram_prob = bigram_probs[bigram]
            else:
                bigram_prob = 1 / (unigram_probs.get(bigram[0], 1/V) * V)
            total_log_prob += math.log(bigram_prob)
    return math.exp(-total_log_prob/N)

In [ ]:
with open("A1_DATASET/train.txt", 'r') as file:
    dev_lines = file.readlines()

sentences_dev = [preprocess(line) for line in dev_lines]
sentences_dev = handle_oov(sentences_dev)

perplexity_unigram_laplace = compute_perplexity_unigram(sentences_dev, unigram_probs_laplace)
perplexity_bigram_laplace = compute_perplexity_bigram(sentences_dev, unigram_probs_laplace, bigram_probs_laplace, V)

perplexity_unigram_kSmooth_0_5 = compute_perplexity_unigram(sentences_dev, unigram_probs_kSmooth_0_5)
perplexity_bigram_kSmooth_0_5 = compute_perplexity_bigram(sentences_dev, unigram_probs_kSmooth_0_5, bigram_probs_kSmooth_0_5, V)

perplexity_unigram_kSmooth = compute_perplexity_unigram(sentences_dev, unigram_probs_kSmooth)
perplexity_bigram_kSmooth = compute_perplexity_bigram(sentences_dev, unigram_probs_kSmooth, bigram_probs_kSmooth, V)

print(f"Unigram Perplexity laplace on train.txt: {perplexity_unigram_laplace}")
print(f"Bigram Perplexity laplace on train.txt: {perplexity_bigram_laplace}")
print("\t")
print(f"Unigram Perplexity kSmooth on train.txt(k=0.5): {perplexity_unigram_kSmooth_0_5}")
print(f"Bigram Perplexity kSmooth on train.txt(k=0.5): {perplexity_bigram_kSmooth_0_5}")
print("\t")
print(f"Unigram Perplexity kSmooth on train.txt(k=0.01): {perplexity_unigram_kSmooth}")
print(f"Bigram Perplexity kSmooth on train.txt(k=0.01): {perplexity_bigram_kSmooth}")

Unigram Perplexity laplace on train.txt: 346.0462261577903
Bigram Perplexity laplace on train.txt: 371.8889027045345
	
Unigram Perplexity kSmooth on train.txt(k=0.5): 345.43397304414805
Bigram Perplexity kSmooth on train.txt(k=0.5): 249.8801101947774
	
Unigram Perplexity kSmooth on train.txt(k=0.01): 345.19665996944235
Bigram Perplexity kSmooth on train.txt(k=0.01): 50.445198314513355


In [ ]:
with open("A1_DATASET/val.txt", 'r') as file:
    dev_lines = file.readlines()

sentences_dev = [preprocess(line) for line in dev_lines]
sentences_dev = handle_oov(sentences_dev)

perplexity_unigram_laplace = compute_perplexity_unigram(sentences_dev, unigram_probs_laplace)
perplexity_bigram_laplace = compute_perplexity_bigram(sentences_dev, unigram_probs_laplace, bigram_probs_laplace, V)

perplexity_unigram_kSmooth_0_5 = compute_perplexity_unigram(sentences_dev, unigram_probs_kSmooth_0_5)
perplexity_bigram_kSmooth_0_5 = compute_perplexity_bigram(sentences_dev, unigram_probs_kSmooth_0_5, bigram_probs_kSmooth_0_5, V)

perplexity_unigram_kSmooth = compute_perplexity_unigram(sentences_dev, unigram_probs_kSmooth)
perplexity_bigram_kSmooth = compute_perplexity_bigram(sentences_dev, unigram_probs_kSmooth, bigram_probs_kSmooth, V)

print(f"Unigram Perplexity laplace: {perplexity_unigram_laplace}")
print(f"Bigram Perplexity laplace: {perplexity_bigram_laplace}")
print("\t")
print(f"Unigram Perplexity kSmooth(k=0.5): {perplexity_unigram_kSmooth_0_5}")
print(f"Bigram Perplexity kSmooth(k=0.5): {perplexity_bigram_kSmooth_0_5}")
print("\t")
print(f"Unigram Perplexity kSmooth(k=0.01): {perplexity_unigram_kSmooth}")
print(f"Bigram Perplexity kSmooth(k=0.01): {perplexity_bigram_kSmooth}")

Unigram Perplexity laplace: 265.08981067825454
Bigram Perplexity laplace: 87.0437813873405
	
Unigram Perplexity kSmooth(k=0.5): 263.02813268951223
Bigram Perplexity kSmooth(k=0.5): 63.633267615398
	
Unigram Perplexity kSmooth(k=0.01): 261.0735540525644
Bigram Perplexity kSmooth(k=0.01): 23.665885352161315
